In [ ]:
import jax, flax, numpy
import flax.linen as nn
import orbax
from notebookinit import *
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import optax
import tiktoken
import lib_kata
from tqdm.auto import tqdm

2023-12-31 01:54:25.048959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 01:54:25.048986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 01:54:25.049870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 01:54:25.830533: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
text_encoded, text_encoder, text_decoder, vocabulary_size = lib_kata.load_dataset_and_tokenize()

vocabulary: 
 ,e,t,o,a,i,h,s,r,n,
,l,d,u,m,y,w,,,c,f,g,b,p,:,k,v,.,',;,?,!,-,j,q,x,z,3,&,$ 
 size: 39 chars
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [ ]:
context_size=256
embedd_features=17
learning_rate=1e-4
hidden_features_per_layer = 512
residual_layer_count = 7


In [ ]:
X_all,y_all = lib_kata.make_training_Xy(text_encoded, context_size=context_size)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.05, shuffle=False, random_state=42)

In [ ]:
# lib_kata.preview_Xy(X=X_train, y=y_train, text_decoder=text_decoder, count=5, offset=0)

In [ ]:
vocabulary_size

39

In [ ]:
# embedder = nn.Embed(num_embeddings=vocabulary_size, features=embedd_features)
# embedder
# embedder_example_param = embedder.init(jax.random.PRNGKey(0), X_train[0])
# example_X = X_train[0].reshape([1,context_size])
# example_X.shape
# tabulate_fn = nn.tabulate(embedder, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
# print(tabulate_fn(example_X))

In [ ]:
X_small,y_small = lib_kata.make_training_Xy(text_encoded[0:2000], context_size=context_size)

example_X=X_small[0].reshape([1,context_size])

example_target_y=y_small[0].reshape([-1,1])
example_target_y_logits = nn.activation.one_hot(example_target_y,vocabulary_size)
# example_target_y_logits
# text_decoder(jnp.argmax(example_target_y_logits).reshape([-1,1]))

In [ ]:
class ResLayer(nn.Module):
    hidden_size: int = 16

    @nn.compact
    def __call__(self, x):
        h = nn.Dense(features=self.hidden_size)(x)
        h = nn.leaky_relu(h)
        y = x+h
        return y

In [ ]:
hfpl = 3
q_input = jnp.ones(shape=(1,hfpl))
q = ResLayer(hidden_size=hfpl)
q_param = q.init(jax.random.PRNGKey(0), q_input)

tabulate_fn = nn.tabulate(q, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(q_input))

                             ResLayer Summary                              
┏━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ path    ┃ module   ┃ inputs       ┃ outputs      ┃ params               ┃
┡━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│         │ ResLayer │ float32[1,3] │ float32[1,3] │                      │
├─────────┼──────────┼──────────────┼──────────────┼──────────────────────┤
│ Dense_0 │ Dense    │ float32[1,3] │ float32[1,3] │ bias: float32[3]     │
│         │          │              │              │ kernel: float32[3,3] │
│         │          │              │              │                      │
│         │          │              │              │ 12 (48 B)            │
├─────────┼──────────┼──────────────┼──────────────┼──────────────────────┤
│         │          │              │        Total │ 12 (48 B)            │
└─────────┴──────────┴──────────────┴──────────────┴──────────────────────┘
                                                                           
                        Total Parameters: 12 (48 B)                        

In [ ]:
z = q.apply(q_param,q_input)
z

Array([[1.2118777 , 1.133064  , 0.99858797]], dtype=float32)

In [ ]:

    
class SimpleNN2(nn.Module):
    # batch size is implied from data
    context_size: int = 8    
    embedd_features: int = 5
    hidden_features_per_layer: int = 16
    vocabulary_size: int = 39
    

    def setup(self):
        # print(self.context_size, self.embedd_features, "->", self.context_size * self.embedd_features,)
        pass

    @nn.compact
    def __call__(self,x):
        # first, embedd the input
        y0 = nn.Embed(name='embedding',num_embeddings=self.vocabulary_size, features=self.embedd_features)(x)
        # flatten the input vector, as there is no conceptual difference between the token's identity and token's feature here:        
        h = y0.reshape([-1,self.context_size*self.embedd_features])
        # lower the hidden size to "hidden_features_per_layer"
        h = nn.Dense(name='neck', features=self.hidden_features_per_layer)(h)
        # proced with the signal processor layers
        for idx_layer in range(residual_layer_count):
            h = ResLayer(hidden_size=self.hidden_features_per_layer)(h)
        
        # final dense layer down to prediction logits. Note that there might be more logits than in the hidden state, that's OK.
        h = nn.Dense(name='layer_final',features=self.vocabulary_size)(h)  
        y = h
        return y
        
        


simpleNN = SimpleNN2(context_size=context_size, embedd_features=embedd_features, hidden_features_per_layer=hidden_features_per_layer)
simpleNN_params = simpleNN.init(jax.random.PRNGKey(0), example_X)

tabulate_fn = nn.tabulate(simpleNN, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))
print(f'{simpleNN.apply(simpleNN_params,example_X).shape=}')
predicted_logits = simpleNN.apply(simpleNN_params,example_X)
predicted_example_token = jnp.argmax(simpleNN.apply(simpleNN_params,example_X), axis=1)
predicted_logits_str = ", ".join([f'{float(predicted_logits[1,x]):0.2f}' for x in range(predicted_logits.shape[1])])
print(predicted_logits_str)
print(f'predicted token: ', predicted_example_token, "-> >>>", text_decoder(predicted_example_token),"<<<")
print(f'   target token: ', example_target_y, f"confidence: {float(predicted_logits[0,predicted_example_token][0]):0.3f}")


                                         SimpleNN2 Summary                                          
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path               ┃ module    ┃ inputs          ┃ outputs           ┃ params                    ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                    │ SimpleNN2 │ int32[1,256]    │ float32[1,39]     │                           │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ embedding          │ Embed     │ int32[1,256]    │ float32[1,256,17] │ embedding: float32[39,17] │
│                    │           │                 │                   │                           │
│                    │           │                 │                   │ 663 (2.7 KB)              │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ neck               │ Dense     │ float32[1,4352] │ float32[1,512]    │ bias: float32[512]        │
│                    │           │                 │                   │ kernel: float32[4352,512] │
│                    │           │                 │                   │                           │
│                    │           │                 │                   │ 2,228,736 (8.9 MB)        │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_0         │ ResLayer  │ float32[1,512]  │ float32[1,512]    │                           │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_0/Dense_0 │ Dense     │ float32[1,512]  │ float32[1,512]    │ bias: float32[512]        │
│                    │           │                 │                   │ kernel: float32[512,512]  │
│                    │           │                 │                   │                           │
│                    │           │                 │                   │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_1         │ ResLayer  │ float32[1,512]  │ float32[1,512]    │                           │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_1/Dense_0 │ Dense     │ float32[1,512]  │ float32[1,512]    │ bias: float32[512]        │
│                    │           │                 │                   │ kernel: float32[512,512]  │
│                    │           │                 │                   │                           │
│                    │           │                 │                   │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_2         │ ResLayer  │ float32[1,512]  │ float32[1,512]    │                           │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_2/Dense_0 │ Dense     │ float32[1,512]  │ float32[1,512]    │ bias: float32[512]        │
│                    │           │                 │                   │ kernel: float32[512,512]  │
│                    │           │                 │                   │                           │
│                    │           │                 │                   │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_3         │ ResLayer  │ float32[1,512]  │ float32[1,512]    │                           │
├────────────────────┼───────────┼─────────────────┼───────────────────┼───────────────────────────┤
│ ResLayer_3/Dense_0 │ Dense     │ float32[1,512]  │ float32[1,512]    │ bias: float32[512]        │
│                    │           │                 │         




simpleNN.apply(simpleNN_params,example_X).shape=(1, 39)
2.58, -1.96, -3.92, 0.78, -4.63, -2.25, 1.02, 1.59, 3.25, 0.90, -3.35, 4.50, 0.64, 0.02, -1.74, -1.94, 0.89, 0.60, -1.98, 1.34, -3.98, -0.48, 0.92, 0.71, 1.85, 1.39, 1.72, -4.20, 3.95, -0.06, -1.69, 0.60, -1.15, 1.11, -0.42, -0.80, 0.71, -2.15, -0.07
predicted token:  [11] -> >>> l <<<
   target token:  [[16]] confidence: 4.505


In [ ]:
example_target_y

array([[16]], dtype=int32)

In [ ]:
# def cross_entropy_loss(logits, labels):
#     """Returns cross-entropy loss."""
#     assert jnp.all(logits.shape==labels.shape)
#     return -jnp.mean(jnp.sum(logits * labels, axis=1))

In [ ]:
# predicted_logits

In [ ]:
predicted_logits.shape

(1, 39)

In [ ]:
example_target_y.shape

(1, 1)

In [ ]:
# demo the library softmax error checker
optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0])

Array([4.5744185], dtype=float32)

In [ ]:
# example loss between the target and predicted
jnp.log10(optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0]))

Array([0.66033584], dtype=float32)

In [ ]:
# I think I am ready to do a training step

In [ ]:
train_minibatch_size = 16*4096
examples_per_epoch = len(X_train) - train_minibatch_size  # how many unique examples of input-output pairs can we produce out of the train-time dataset
batches_per_epoch =  len(X_train) // train_minibatch_size
print(f'batches_per_epoch per epoch: {batches_per_epoch} with minibatch size of {train_minibatch_size}')
starter_X_minibatch = jnp.array(X_train[0:train_minibatch_size])
starter_y_minibatch = jnp.array(X_train[0:train_minibatch_size])

simpleNN_minibatched = SimpleNN2(context_size=context_size, embedd_features=embedd_features, hidden_features_per_layer=hidden_features_per_layer)
simpleNN_minibatched_params = simpleNN.init(jax.random.PRNGKey(0), starter_X_minibatch)
tabulate_fn = nn.tabulate(simpleNN_minibatched, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(starter_X_minibatch))

batches_per_epoch per epoch: 16 with minibatch size of 65536


                                             SimpleNN2 Summary                                              
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path               ┃ module    ┃ inputs              ┃ outputs               ┃ params                    ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                    │ SimpleNN2 │ int32[65536,256]    │ float32[65536,39]     │                           │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ embedding          │ Embed     │ int32[65536,256]    │ float32[65536,256,17] │ embedding: float32[39,17] │
│                    │           │                     │                       │                           │
│                    │           │                     │                       │ 663 (2.7 KB)              │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ neck               │ Dense     │ float32[65536,4352] │ float32[65536,512]    │ bias: float32[512]        │
│                    │           │                     │                       │ kernel: float32[4352,512] │
│                    │           │                     │                       │                           │
│                    │           │                     │                       │ 2,228,736 (8.9 MB)        │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_0         │ ResLayer  │ float32[65536,512]  │ float32[65536,512]    │                           │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_0/Dense_0 │ Dense     │ float32[65536,512]  │ float32[65536,512]    │ bias: float32[512]        │
│                    │           │                     │                       │ kernel: float32[512,512]  │
│                    │           │                     │                       │                           │
│                    │           │                     │                       │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_1         │ ResLayer  │ float32[65536,512]  │ float32[65536,512]    │                           │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_1/Dense_0 │ Dense     │ float32[65536,512]  │ float32[65536,512]    │ bias: float32[512]        │
│                    │           │                     │                       │ kernel: float32[512,512]  │
│                    │           │                     │                       │                           │
│                    │           │                     │                       │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_2         │ ResLayer  │ float32[65536,512]  │ float32[65536,512]    │                           │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_2/Dense_0 │ Dense     │ float32[65536,512]  │ float32[65536,512]    │ bias: float32[512]        │
│                    │           │                     │                       │ kernel: float32[512,512]  │
│                    │           │                     │                       │                           │
│                    │           │                     │                       │ 262,656 (1.1 MB)          │
├────────────────────┼───────────┼─────────────────────┼───────────────────────┼───────────────────────────┤
│ ResLayer_3         │ ResLayer  │ float32[65536,512]  │ float32[65536,512] 

In [ ]:
model_param_count = numpy.sum(numpy.array([numpy.prod(x.shape) for x in jax.tree_util.tree_leaves(simpleNN_minibatched_params)]))
model_size_str = f'RN_{model_param_count/1e3:0.1f}k'
print(f'model size: {model_size_str}')
train_data_size = len(X_train)
print(f'model size to dataset size ratio: {model_param_count/train_data_size:0.3f}')

model size: RN_4088.0k
model size to dataset size ratio: 3.859


In [ ]:
X_train=jnp.array(X_train)

In [ ]:
y_train=jnp.array(y_train)

In [ ]:
y_train.shape

(1059381,)

In [ ]:
X_train.shape

(1059381, 256)

In [ ]:
def prep_minibatch(minibatch_idx=0):
    minibatch_ptr = minibatch_idx*train_minibatch_size
    X_minibatch = jnp.array(X_train[minibatch_ptr:minibatch_ptr+train_minibatch_size,:])
    y_minibatch = jnp.array(y_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch_target_logits = nn.activation.one_hot(y_minibatch,vocabulary_size)    
    return X_minibatch, y_minibatch, y_minibatch_target_logits
minibatch_idx = 4
minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)

In [ ]:
%timeit minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)

1.16 ms ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
print(y_minibatch_target_logits.devices())

{cuda(id=0)}


In [ ]:
# debug the X/y pair correctness
# for idx_step in range(3):
#     print(minibatch_X[idx_step,:], "->", minibatch_y[idx_step])

In [ ]:
model_starter_params = simpleNN_minibatched_params
model_moving_params = model_starter_params.copy()
optimizer = optax.adabelief(learning_rate=learning_rate)
optimizer_state = optimizer.init(model_moving_params)
step_count=0
minibatch_idx = 0
examples_seen = 0

In [ ]:
import wandb

In [ ]:
wandb.finish()

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="04_benchmark_perceptron",    
    name=model_size_str,
    # track hyperparameters and run metadata
    config={
        'model_param_count':model_param_count,
        "examples_per_epoch":examples_per_epoch,
        "batches_per_epoch":batches_per_epoch,
        "context_size":context_size,
        "embedd_features":embedd_features,
        "learning_rate": learning_rate,
        "layer_count": residual_layer_count,
        "type" : "resnet"
    }
    )

wandb: Currently logged in as: greygoo. Use `wandb login --relogin` to force relogin


In [ ]:
# attempt validation accuracy
@jax.jit
def validate(model_params, validation_size=8*4096):
    validation_expected_tokens = jnp.array(y_test[0:validation_size])
    validation_predicted_logits = simpleNN_minibatched.apply(model_params, jnp.array(X_test[0:validation_size]))
    validation_loss = jnp.mean(optax.softmax_cross_entropy_with_integer_labels(validation_predicted_logits, validation_expected_tokens))
    predicted_tokens = jnp.argmax(validation_predicted_logits, axis=1)
    accurate_tokens = jnp.sum(predicted_tokens==validation_expected_tokens)
    accuracy_ratio = accurate_tokens/validation_size
    return validation_loss, accuracy_ratio

q=validate(model_starter_params)
q

(Array(5.9921494, dtype=float32), Array(0.03173828, dtype=float32))

In [ ]:
loss_fn = lambda model_params, X,y : jnp.mean(optax.softmax_cross_entropy_with_integer_labels(simpleNN_minibatched.apply(model_params,X),  y))
jitted_val_and_grad = jax.jit(jax.value_and_grad(loss_fn, argnums=0))

In [ ]:
q[0].devices()

{cuda(id=0)}

In [ ]:
%timeit validate(model_starter_params)

49.2 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit loss_value, gradients = jitted_val_and_grad(model_moving_params, minibatch_X, minibatch_y)  

107 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)

1.18 ms ± 66.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
minibatch_X.devices()

{cuda(id=0)}

In [ ]:
# training loop:
# epochs, and inside, batches

for idx_epoch in tqdm(range(1000), desc="epoch"):
    for idx_minibatch in tqdm(range(batches_per_epoch), leave=False,  desc="batch in epoch" ):
        step_count+=1
        minibatch_idx = step_count % batches_per_epoch        
        minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)        
        loss_value, gradients = jitted_val_and_grad(model_moving_params, minibatch_X, minibatch_y)        
        model_param_updates, optimizer_state = optimizer.update(gradients, optimizer_state, params=model_moving_params)
        model_moving_params = optax.apply_updates(model_moving_params, model_param_updates)
        examples_seen += train_minibatch_size
        
        validation_loss, validation_accuracy_ratio = validate(model_moving_params)
        wandb.log({
            "step_count":step_count, 
            "dataset_seen_ratio":examples_seen/examples_per_epoch, 
            "train_loss_value": loss_value,            
            "validation_loss":validation_loss, 
            "validation_accuracy_ratio":validation_accuracy_ratio,
            })
        

# after training, generate example text with a prompt from the validation data.

starter_prompt = jnp.array(X_train[0])

def predict_text(model_params, prompt_encoded, new_characters=250):
    prediction_decoded = text_decoder(prompt_encoded)
    print(f'{prediction_decoded}')
    print(f'-------------------------------------------------------')
    running_prompt = prompt_encoded.copy()
    continuation = ""
    # print(running_prompt)
    for char_idx in range(new_characters):
        predicted_logits = simpleNN_minibatched.apply(model_params, running_prompt)
        predicted_token = jnp.argmax(predicted_logits).reshape((1,1))
        predicted_char = text_decoder(predicted_token)
        continuation = f'{continuation}{predicted_char}'
        running_prompt = jnp.hstack([running_prompt[1:],predicted_token[:,0]])
        # print(running_prompt)
    return continuation  
example_generation = predict_text(model_moving_params, starter_prompt)
print(example_generation)
wandb.log({'example_generation_html':wandb.Html(example_generation)})
wandb.finish()


epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

...

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

batch in epoch:   0%|          | 0/16 [00:00<?, ?it/s]

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:

-------------------------------------------------------
we know't, we know't.

first citizen:
let us kill him, and we'll have corn at our own price.
is't a verdict?

all:
no more talking on't; let it be done: away, away!

second citizen:
one word, good citizens.

first citizen:
we are accounted poor citiz


dataset_seen_ratio,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
step_count,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss_value,█▆▅▅▄▄▄▃▄▂▂▃▂▂▂▂▁▂▁▁▁▁▂▂▁▁▂▂▁▁▂▁▁▁▁▂▁▁▁▁
validation_accuracy_ratio,▁▇██▇▆▅▅▆▄▅▄▄▅▄▄▄▄▄▄▃▄▄▅▅▄▄▄▄▃▄▄▄▃▄▃▃▄▄▄
validation_loss,▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▆▆▆▆▇▅▇▆▆█▇▇██▇█▇█▇▇█
dataset_seen_ratio,1055.06996
step_count,16000
train_loss_value,0.00395
validation_accuracy_ratio,0.28189
validation_loss,11.5342


In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:
!git add .
!git commit -m 'savegame. now appears to train.'
!git push

In [ ]:
#starter_prompt = jnp.array(X_train[0])
starter_prompt = jnp.array(X_test[0])

def predict_text(model_params, prompt_encoded, new_characters=1500):
    prediction_decoded = text_decoder(prompt_encoded)
    print(f'{prediction_decoded}')
    print(f'-------------------------------------------------------')
    running_prompt = prompt_encoded.copy()
    continuation = ""
    # print(running_prompt)
    for char_idx in range(new_characters):
        predicted_logits = simpleNN_minibatched.apply(model_params, running_prompt)
        predicted_token = jnp.argmax(predicted_logits).reshape((1,1))
        predicted_char = text_decoder(predicted_token)
        print(f'{predicted_char}',end='')
        continuation = f'{continuation}{predicted_char}'
        running_prompt = jnp.hstack([running_prompt[1:],predicted_token[:,0]])
        # print(running_prompt)
    return continuation  
example_generation = predict_text(model_moving_params, starter_prompt)
# print(example_generation)

t like a father you will deal with him
and pass my daughter a sufficient dower,
the match is made, and all is done:
your son shall have my daughter with consent.

tranio:
i thank you, sir. where then do you know best
we be affied and such assurance ta'en
a
-------------------------------------------------------
ndeed;
weat! at need, andy yos be thun that whea,
not siok to borran.

hang an breaposan ata,
for-powh cand, now toer ousu beat for wielf:
yhat out that haverisyspeor fith wim
srongeruchat.

caminius, aft my pore therds what have dots tim hear, tore of with her covberlabe, do think sean thou there yeart of thath enss upood up they hencly for tient thanff
ad, ef and and spiliss, chave!
thit i morld by deams,
that thath's is he whath ring have be hepry
to my darge-poon are, thy and thin thencl wordung;
in
word blius efewill o bat give.
sorre don:
sence:
and manger ase apetorret sonew bab weres?

yrush in fairs, by and cuth thes well of and en thou hold be pled.

groriousen:
marrif